# 유사도 모델

## 모델 선정

- sample 데이터를 이용한 임계값 보정

### jhgan/ko-sroberta-multitask 모델

In [ ]:
from sentence_transformers import SentenceTransformer, models
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, precision_recall_fscore_support, roc_auc_score
import numpy as np, pandas as pd, torch

path="/content/drive/MyDrive/낚시성기사/샘플뉴스데이터.csv"
backbone="jhgan/ko-sroberta-multitask"
df=pd.read_csv(path).dropna(subset=["title","summary","label"]).reset_index(drop=True)

word=models.Transformer(backbone)
pool=models.Pooling(word.get_word_embedding_dimension(), pooling_mode_mean_tokens=True, pooling_mode_cls_token=False, pooling_mode_max_tokens=False)
m=SentenceTransformer(modules=[word, pool], device="cuda" if torch.cuda.is_available() else "cpu")

trval_idx, te_idx = train_test_split(np.arange(len(df)), test_size=0.2, random_state=42, stratify=df["label"])
df_trval=df.iloc[trval_idx].reset_index(drop=True)
df_te=df.iloc[te_idx].reset_index(drop=True)

E1_trval=m.encode(df_trval["title"].tolist(), normalize_embeddings=True, batch_size=min(256,len(df_trval)), convert_to_numpy=True, show_progress_bar=False)
E2_trval=m.encode(df_trval["summary"].tolist(), normalize_embeddings=True, batch_size=min(256,len(df_trval)), convert_to_numpy=True, show_progress_bar=False)
S_trval=np.sum(E1_trval*E2_trval, axis=1)
Y_trval=df_trval["label"].astype(int).values

E1_te=m.encode(df_te["title"].tolist(), normalize_embeddings=True, batch_size=min(256,len(df_te)), convert_to_numpy=True, show_progress_bar=False)
E2_te=m.encode(df_te["summary"].tolist(), normalize_embeddings=True, batch_size=min(256,len(df_te)), convert_to_numpy=True, show_progress_bar=False)
S_te=np.sum(E1_te*E2_te, axis=1)
Y_te=df_te["label"].astype(int).values

def fbeta_at_t(y_true, s, t, beta=1.0):
    pred=(s<t).astype(int)
    p,r,f,_=precision_recall_fscore_support(y_true, pred, average="binary", zero_division=0)
    if beta==1.0: return f
    b2=beta*beta
    if p+r==0: return 0.0
    return (1+b2)*(p*r)/(b2*p+r+1e-12)

skf=StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
Ts=[]; val_scores=[]
beta=0.5
for tr_idx, va_idx in skf.split(S_trval, Y_trval):
    sv=S_trval[va_idx]; yv=Y_trval[va_idx]
    svu=np.unique(np.sort(sv))
    edges=(svu[:-1]+svu[1:])/2.0
    ths=np.concatenate(([svu[0]-1e-6], edges, [svu[-1]+1e-6]))
    scores=[fbeta_at_t(yv, sv, t, beta=beta) for t in ths]
    t=ths[int(np.argmax(scores))]
    Ts.append(float(t))
    val_scores.append(float(np.max(scores)))

t_cv=float(np.median(Ts))
pred_te=(S_te<t_cv).astype(int)
p,r,f,_=precision_recall_fscore_support(Y_te, pred_te, average="binary", zero_division=0)
auc=roc_auc_score(Y_te, -S_te)

np.savez("/content/drive/MyDrive/낚시성기사/koclickbait_threshold_cv_holdout.npz", t=t_cv, model_name=backbone, folds=np.array(Ts), val_scores=np.array(val_scores), beta=beta)
print({"n_total":int(len(df)),"n_trval":int(len(df_trval)),"n_test":int(len(df_te)),"beta":beta,"t_cv":t_cv,"val_score_mean":float(np.mean(val_scores)),"val_score_std":float(np.std(val_scores,ddof=1)),"test_precision":float(p),"test_recall":float(r),"test_f1":float(f),"test_auc":float(auc)})

config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

{'n_total': 1000, 'n_trval': 800, 'n_test': 200, 'beta': 0.5, 't_cv': 0.14186015725135803, 'val_score_mean': 0.5654806286720528, 'val_score_std': 0.034501663402087976, 'test_precision': 0.5784313725490197, 'test_recall': 0.6555555555555556, 'test_f1': 0.6145833333333334, 'test_auc': 0.6082828282828283}


### snunlp/KR-SBERT-V40K-klueNLI-augSTS 모델

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, precision_recall_fscore_support, roc_auc_score
import numpy as np, pandas as pd, torch

path="/content/drive/MyDrive/낚시성기사/샘플뉴스데이터.csv"
model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS"
df=pd.read_csv(path).dropna(subset=["title","summary","label"]).reset_index(drop=True)

m=SentenceTransformer(model_name, device="cuda" if torch.cuda.is_available() else "cpu")
idx=np.arange(len(df)); y=df["label"].astype(int).values
trval_idx, te_idx=train_test_split(idx, test_size=0.2, random_state=42, stratify=y)
trval=df.iloc[trval_idx].reset_index(drop=True); te=df.iloc[te_idx].reset_index(drop=True)

E1=m.encode(trval["title"].tolist(), normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=False)
E2=m.encode(trval["summary"].tolist(), normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=False)
S_trval=np.sum(E1*E2, axis=1); Y_trval=trval["label"].astype(int).values

E1=m.encode(te["title"].tolist(), normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=False)
E2=m.encode(te["summary"].tolist(), normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=False)
S_te=np.sum(E1*E2, axis=1); Y_te=te["label"].astype(int).values

skf=StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
Ts=[]; scores=[]
for tr_idx, va_idx in skf.split(S_trval, Y_trval):
    sv=S_trval[va_idx]; yv=Y_trval[va_idx]
    svu=np.unique(np.sort(sv)); edges=(svu[:-1]+svu[1:])/2.0
    ths=np.concatenate(([svu[0]-1e-6], edges, [svu[-1]+1e-6]))
    fs=[f1_score(yv,(sv<t).astype(int)) for t in ths]
    Ts.append(float(ths[int(np.argmax(fs))])); scores.append(float(np.max(fs)))
t=float(np.median(Ts))
pred=(S_te<t).astype(int)
p,r,f,_=precision_recall_fscore_support(Y_te, pred, average="binary")
auc=roc_auc_score(Y_te, -S_te)
print({"t":t,"val_f1_mean":float(np.mean(scores)),"test_precision":float(p),"test_recall":float(r),"test_f1":float(f),"test_auc":float(auc)})

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/467M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

{'t': 0.5740045309066772, 'val_f1_mean': 0.6517553413282664, 'test_precision': 0.48128342245989303, 'test_recall': 1.0, 'test_f1': 0.6498194945848376, 'test_auc': 0.676060606060606}


- 이 경우 threshold 값이 너무 높아 전부 낚시 데이터로 찍고 있는 상태
- 다음 실험으로 검증 세트에서 원하는 정밀도 이상을 보장하는 임계값으로 재튜닝

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
import numpy as np, pandas as pd, torch

path="/content/drive/MyDrive/낚시성기사/샘플뉴스데이터.csv"
model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS"
df=pd.read_csv(path).dropna(subset=["title","summary","label"]).reset_index(drop=True)

m=SentenceTransformer(model_name, device="cuda" if torch.cuda.is_available() else "cpu")
idx=np.arange(len(df)); y=df["label"].astype(int).values
trval_idx, te_idx=train_test_split(idx, test_size=0.2, random_state=42, stratify=y)
trval=df.iloc[trval_idx].reset_index(drop=True); te=df.iloc[te_idx].reset_index(drop=True)

e1=m.encode(trval["title"].tolist(), normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=False)
e2=m.encode(trval["summary"].tolist(), normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=False)
S_trval=np.sum(e1*e2, axis=1); Y_trval=trval["label"].astype(int).values

e1=m.encode(te["title"].tolist(), normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=False)
e2=m.encode(te["summary"].tolist(), normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=False)
S_te=np.sum(e1*e2, axis=1); Y_te=te["label"].astype(int).values

skf=StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
Ts=[]
for _, va_idx in skf.split(S_trval, Y_trval):
    sv=S_trval[va_idx]; yv=Y_trval[va_idx]
    svu=np.unique(np.sort(sv)); edges=(svu[:-1]+svu[1:])/2.0
    ths=np.concatenate(([svu[0]-1e-6], edges, [svu[-1]+1e-6]))
    best=None; best_r=-1.0
    for t in ths:
        pred=(sv<t).astype(int)
        p,r,f,_=precision_recall_fscore_support(yv, pred, average="binary", zero_division=0)
        if p>=0.70 and r>best_r:
            best_r=r; best=t; best_f=f
    if best is None:
        fs=[]
        for t in ths:
            pred=(sv<t).astype(int)
            _,_,f,_=precision_recall_fscore_support(yv, pred, average="binary", zero_division=0)
            fs.append(f)
        best=ths[int(np.argmax(fs))]
    Ts.append(float(best))

t=float(np.median(Ts))
pred=(S_te<t).astype(int)
p,r,f,_=precision_recall_fscore_support(Y_te, pred, average="binary", zero_division=0)
auc=roc_auc_score(Y_te, -S_te)
print({"t":t,"test_precision":float(p),"test_recall":float(r),"test_f1":float(f),"test_auc":float(auc)})

{'t': 0.06432147324085236, 'test_precision': 0.7674418604651163, 'test_recall': 0.36666666666666664, 'test_f1': 0.49624060150375937, 'test_auc': 0.676060606060606}


### 두 모델의 앙상블

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, precision_recall_fscore_support, roc_auc_score
import numpy as np, pandas as pd, torch

path="/content/drive/MyDrive/낚시성기사/샘플뉴스데이터.csv"
m1=SentenceTransformer("jhgan/ko-sroberta-multitask", device="cuda" if torch.cuda.is_available() else "cpu")
m2=SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS", device="cuda" if torch.cuda.is_available() else "cpu")
df=pd.read_csv(path).dropna(subset=["title","summary","label"]).reset_index(drop=True)
idx=np.arange(len(df)); y=df["label"].astype(int).values
trval_idx, te_idx=train_test_split(idx, test_size=0.2, random_state=42, stratify=y)
trval=df.iloc[trval_idx].reset_index(drop=True); te=df.iloc[te_idx].reset_index(drop=True)

def simpair(m, a, b):
    e1=m.encode(a, normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=False)
    e2=m.encode(b, normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=False)
    return np.sum(e1*e2, axis=1)

S1_trval=simpair(m1, trval["title"].tolist(), trval["summary"].tolist())
S2_trval=simpair(m2, trval["title"].tolist(), trval["summary"].tolist())
S_trval=(S1_trval+S2_trval)/2.0; Y_trval=trval["label"].astype(int).values

S1_te=simpair(m1, te["title"].tolist(), te["summary"].tolist())
S2_te=simpair(m2, te["title"].tolist(), te["summary"].tolist())
S_te=(S1_te+S2_te)/2.0; Y_te=te["label"].astype(int).values

skf=StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
Ts=[]; scores=[]
for tr_idx, va_idx in skf.split(S_trval, Y_trval):
    sv=S_trval[va_idx]; yv=Y_trval[va_idx]
    svu=np.unique(np.sort(sv)); edges=(svu[:-1]+svu[1:])/2.0
    ths=np.concatenate(([svu[0]-1e-6], edges, [svu[-1]+1e-6]))
    fs=[f1_score(yv,(sv<t).astype(int)) for t in ths]
    Ts.append(float(ths[int(np.argmax(fs))])); scores.append(float(np.max(fs)))
t=float(np.median(Ts))
pred=(S_te<t).astype(int)
p,r,f,_=precision_recall_fscore_support(Y_te, pred, average="binary")
auc=roc_auc_score(Y_te, -S_te)
print({"t":t,"val_f1_mean":float(np.mean(scores)),"test_precision":float(p),"test_recall":float(r),"test_f1":float(f),"test_auc":float(auc)})

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

{'t': 0.5735927820205688, 'val_f1_mean': 0.6493346587313723, 'test_precision': 0.48128342245989303, 'test_recall': 1.0, 'test_f1': 0.6498194945848376, 'test_auc': 0.6633333333333333}


- 앙상블이 recall=1.0, precision≈0.48인 건 두 모델 유사도 스케일이 달라서 평균했더니 임계값이 한쪽으로 치우친 상태

- 다양한 모델을 사용한 이유: 각 모델이 문장 의미를 다른 기준으로 인코딩하기 때문
- 앙상블을 사용한 이유 : 서로 다른 임베딩이 포착한 의미 차이를 평균·학습으로 통합
- 하지만 결론적으로 이 방법은 성능이 좋지 않음
- 따라서 모델만 바꾸는 것이 아니라 사전 학습된 모델을 파인 튜닝하는 과정이 필요

- 단순히 모델을 바꾸고 조합하는 것만으로는 낚시 기사의 '미묘함'을 학습하지 못하므로 '대조학습을 통한 파인 튜닝'이 필요

## 최종 모델

In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/낚시성기사/new_summarization30000.csv")
df.head(n=5)

,doc_id,newsTitle,newsContent,clickbaitClass,summary
0,EC_M02_000008,2017 다우존스 지속가능경영지수 어떤 기업이 편입되었나?,"런던 현지 시각으로 7일, 스탠다드앤푸어스 다우존스 인디시즈와 로베코샘이 다우존스지...",0,올해 국내 기업으로는 LG전자가 국내 유일하게 '인더스트리 그룹 리더'에 4년 연속...
1,EC_M02_000011,"한국교직원공제회, 13일 스튜어드십 코드 도입, 사회책임투자 전면 시행 결정",한국교직원공제회는 13일 운영위원회를 열어 스튜어드십 코드를 도입하기로 결정했다.\...,0,한국교직원공제회는 13일 운영위원회를 열어 총자산 30조 원대 규모의 교직원공제회의...
2,EC_M02_000015,기업 규모에 따라 CSR 공시 연착륙 필요 있어,"\""기업의 규모, 특성에 따라 CSR 정보 공시 내용을 의무와 권고로 나눠 기업들이...",0,지난 18일 국회 정무위원회 업무보고에서 홍일표 자유한국당 의원이 최흥식 금융감독원...
3,EC_M02_000021,"공기업 등, 노인 손 감각 유지 위한 점핑클레이로 사회공헌",공기업 등이 노인의 손 감각 유지에 도움이 되는 점핑 클레이로 사회 공헌 활동을 벌...,0,한국전력 서울지역본부 사회봉사단은 지난 19일 서울 종로구 치매노인데이케어센터에서 ...
4,EC_M02_000024,"하이자산운용, 자산운용사 가운데 두번째로 `스튜어드십 코드` 도입!",자산운용사들이 연이어 스튜어드십코드를 도입하고 있다.\n하이자산운용은3일스튜어드십코...,0,하이자산운용은 지난 9월 한국기업지배구조원에 따르면 현재스튜어드십코드를 도입하겠다고...


### SBERT --> 대조학습

In [ ]:
import os, warnings, json, random
os.environ["WANDB_MODE"]="disabled"
os.environ["TOKENIZERS_PARALLELISM"]="false"
warnings.filterwarnings("ignore")

import numpy as np, pandas as pd, torch
from sentence_transformers import SentenceTransformer, losses, InputExample
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_recall_fscore_support, roc_auc_score

def set_seed(s=42):
    random.seed(s); np.random.seed(s); torch.manual_seed(s); torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic=True; torch.backends.cudnn.benchmark=False

set_seed(42)

path="/content/drive/MyDrive/낚시성기사/new_summarization30000.csv"
backbone="snunlp/KR-SBERT-V40K-klueNLI-augSTS"
out_dir="/content/drive/MyDrive/낚시성기사/kosimcse_ft_contrastive_single"

# --- 데이터 준비 ---
df=pd.read_csv(path).dropna(subset=["newsTitle","summary","clickbaitClass"]).reset_index(drop=True)
y_clickbait=(df["clickbaitClass"].astype(int)==0).astype(int)
idx=np.arange(len(df))
tr_idx, te_idx=train_test_split(idx, test_size=0.2, random_state=42, stratify=y_clickbait)
tr_df=df.iloc[tr_idx].reset_index(drop=True); te_df=df.iloc[te_idx].reset_index(drop=True)

y_tr=(tr_df["clickbaitClass"].astype(int)==0).astype(int)
tr2_idx, va_idx=train_test_split(np.arange(len(tr_df)), test_size=0.2, random_state=42, stratify=y_tr)
train_df=tr_df.iloc[tr2_idx].reset_index(drop=True); val_df=tr_df.iloc[va_idx].reset_index(drop=True)

pos_mask=train_df.clickbaitClass.astype(int)==1
neg_mask=~pos_mask
train_examples=[InputExample(texts=[a,b], label=1.0) for a,b in zip(train_df.loc[pos_mask,"newsTitle"], train_df.loc[pos_mask,"summary"])]
train_examples+=[InputExample(texts=[a,b], label=0.0) for a,b in zip(train_df.loc[neg_mask,"newsTitle"], train_df.loc[neg_mask,"summary"])]

dl=DataLoader(train_examples, batch_size=max(1,min(64,len(train_examples))), shuffle=True, drop_last=True)

# --- 이어서 학습하기 ---
if os.path.exists(os.path.join(out_dir, "config.json")):
    print("🔄 기존 체크포인트 감지됨 — 이어서 학습합니다.")
    model=SentenceTransformer(out_dir, device="cuda" if torch.cuda.is_available() else "cpu")
else:
    print("🚀 새로운 학습 시작")
    model=SentenceTransformer(backbone, device="cuda" if torch.cuda.is_available() else "cpu")

loss_fn=losses.ContrastiveLoss(model, margin=0.2)

# 에폭 1회 단위로 저장 (중간 끊김 대비)
model.fit(
    train_objectives=[(dl,loss_fn)],
    epochs=5,
    warmup_steps=int(0.1*len(dl)*5),
    output_path=out_dir,
    checkpoint_path=os.path.join(out_dir, "checkpoints"),
    checkpoint_save_steps=len(dl),  # 한 에폭마다
    checkpoint_save_total_limit=1,  # 마지막만 유지
    show_progress_bar=True
)

# --- 평가 ---
best=SentenceTransformer(out_dir, device="cuda" if torch.cuda.is_available() else "cpu")

e1=best.encode(val_df.newsTitle.tolist(), normalize_embeddings=True, convert_to_numpy=True, batch_size=256, show_progress_bar=False)
e2=best.encode(val_df.summary.tolist(), normalize_embeddings=True, convert_to_numpy=True, batch_size=256, show_progress_bar=False)
s=np.sum(e1*e2,axis=1)
yv=(val_df.clickbaitClass.astype(int)==0).astype(int).values
sv=np.unique(np.sort(s)); edges=(sv[:-1]+sv[1:])/2.0; ths=np.concatenate(([sv[0]-1e-6],edges,[sv[-1]+1e-6]))
t=float(ths[int(np.argmax([f1_score(yv,(s<tt).astype(int)) for tt in ths]))])

os.makedirs(out_dir, exist_ok=True)
with open(os.path.join(out_dir,"best_threshold.json"),"w",encoding="utf-8") as f:
    json.dump({"threshold": t}, f, ensure_ascii=False)

e1=best.encode(te_df.newsTitle.tolist(), normalize_embeddings=True, convert_to_numpy=True, batch_size=256, show_progress_bar=False)
e2=best.encode(te_df.summary.tolist(), normalize_embeddings=True, convert_to_numpy=True, batch_size=256, show_progress_bar=False)
s=np.sum(e1*e2,axis=1)
yt=(te_df.clickbaitClass.astype(int)==0).astype(int).values
pred=(s<t).astype(int)
p,r,f,_=precision_recall_fscore_support(yt,pred,average="binary",zero_division=0)
auc=roc_auc_score(yt,-s)
print({"t":t,"precision":float(p),"recall":float(r),"f1":float(f),"auc":float(auc)})

🔄 기존 체크포인트 감지됨 — 이어서 학습합니다.


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,0.001600
1000,0.000800
1500,0.000500


{'t': 0.9402652382850647, 'precision': 0.7752834467120181, 'recall': 0.8873604983130029, 'f1': 0.8275444753721408, 'auc': 0.8291894849886011}


### 모델 불러오기

In [2]:
from sentence_transformers import SentenceTransformer
import numpy as np
import json

model_path="/content/drive/MyDrive/낚시성기사/kosimcse_ft_contrastive_single"
model = SentenceTransformer(model_path)

# threshold 불러오기
with open(f"{model_path}/best_threshold.json", "r", encoding="utf-8") as f:
    threshold = json.load(f)["threshold"]
    f1_score = json.load(f)['f1']

print("Loaded threshold:", threshold)

Loaded threshold: 0.9402652382850647


In [21]:
# 단일 기사 테스트 함수
def predict_clickbait(title, summary, model, threshold):
    e1 = model.encode([title], normalize_embeddings=True)
    e2 = model.encode([summary], normalize_embeddings=True)

    sim = float(np.sum(e1[0] * e2[0]))  # 코사인 유사도

    pred = int(sim < threshold)  # 1=낚시, 0=정상

    return {
        "similarity": sim,
        "threshold": threshold,
        "pred": pred,
        "label": "낚시성 기사" if pred == 1 else "정상 기사"
    }

# 출력값 조정
def print_prediction(title, summary, model, threshold, f1_score):
    result = predict_clickbait(title, summary, model, threshold)

    print("===== 단일 기사 테스트 결과 =====\n")
    print(f"요약문:")
    print(f"{summary}\n")

    print(f"코사인 유사도: {result['similarity']:.4f}")
    print(f"컷오프(Threshold): {result['threshold']}\n")

    print(f"예측 결과: {result['label']}")
    print(f"현재 모델 F1-score: {f1_score * 100:.2f}%")

    return {
        "summary": summary,
        "threshold": threshold,
        "prediction": result["label"],
        "f1_score_percent": round(f1_score * 100, 2)
    }

In [19]:
# 테스트 1
pred = print_prediction(
title = "[종합] 김영옥, 심근경색으로 사망…검찰청서 쓰러졌다 ('야한사진관')",
summary = "권나라의 하나 밖에 없는 가족 김영옥이 사망해 주원의 사진관에 귀객으로 찾아왔다.",
    model = model,
    threshold = threshold,
    f1_score = 0.83
)

===== 단일 기사 테스트 결과 =====

요약문:
권나라의 하나 밖에 없는 가족 김영옥이 사망해 주원의 사진관에 귀객으로 찾아왔다.

코사인 유사도: 0.4782
컷오프(Threshold): 0.9402652382850647

예측 결과: 낚시성 기사
현재 모델 F1-score: 83.00%


In [22]:
# 테스트 2
pred = print_prediction(
title = "MC몽 “한달 전 극단적 선택…이젠 정말 강하게 살겠다",
summary = "MC몽은 11일 인스타그램에 상처 난 손목 사진을 올리고 “한 달 전 극단적 선택을 시도했다고 밝혔다.",
    model=model,
    threshold = threshold,
    f1_score = 0.83
)

===== 단일 기사 테스트 결과 =====

요약문:
MC몽은 11일 인스타그램에 상처 난 손목 사진을 올리고 “한 달 전 극단적 선택을 시도했다고 밝혔다.

코사인 유사도: 0.9086
컷오프(Threshold): 0.9402652382850647

예측 결과: 낚시성 기사
현재 모델 F1-score: 83.00%
